До начала Вам надо будет установить библиотеку datasets.<br>Для этого прописываем команду в anaconda_promt: **conda install -c conda-forge datasets**<br> Далее ждем загрузки и запускаем код ниже. Если выдает ошибку, попробуйте выполнить еще одну команду:<br>**conda install -c conda-forge pyarrow**<br>Должно помочь.

In [2]:
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from datasets import load_dataset
%matplotlib inline

if tf.config.list_logical_devices("GPU"):
    print("CUDA is available!")
else:
    print("CPU will be used!")

CUDA is available!


In [3]:
#Загрузим датасет
dataset = load_dataset("MonoHime/ru_sentiment_dataset")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'sentiment'],
        num_rows: 189891
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'sentiment'],
        num_rows: 21098
    })
})


In [4]:
dataset_train, dataset_val = dataset["train"], dataset["validation"]
dataset_train = dataset_train.to_list()
dataset_val = dataset_val.to_list()

for x in dataset_val[:5]:
    print("Text:\n{}\nLabel(0-Neutral, 1-Positive, 2-Negative):\n{}".format(x["text"],x["sentiment"]))

Text:
Развода на деньги нет Наблюдаюсь в Лайфклиник по беременности, развода на деньги нет, врачи не плохие, по поводу ресепшен соглашусь (путают документы). 
Label(0-Neutral, 1-Positive, 2-Negative):
1
Text:
Отель выбрали потому что рядом со стадионом. Отель 4*. Номер большой. Кровать 2-спальная одна. 2 одеяла. Много подушек. Есть зона отдыха. Чайный сет. 2 бутылки по 0,5 л бесплатно каждый день. Много шкафов. Мини-бар. Утюг и гладильная доска, отдельно прибор для глажки брюк. Санузел общий большой. Ванна, лейка съемная. 2 раковины, фен. Халаты, тапочки. Расширенный пакет косметических средств, даже соль для ванны. Интернет быстрый. Так как этаж высокий был, вид из окна на город. Один недостаток в номере 919: он напротив хозлифта и с утра начинает пользоваться им персонал для обслуживания завтрака на 9 этаже. Очень слышно звон посуды и разговоры. Завтрак не включен, стоит 23 евро. Рядом с отелем есть кафе и subway.
Label(0-Neutral, 1-Positive, 2-Negative):
0
Text:
Вылечили Гноился с р

In [5]:
dataset_train = [(" ".join(x["text"].split(" ")[:50]), x["sentiment"]) for x in dataset_train][:50000]
dataset_val = [(" ".join(x["text"].split(" ")[:50]), x["sentiment"]) for x in dataset_val][:10000]
texts = list(list(zip(*dataset_train))[0]) + list(list(zip(*dataset_val))[0])
texts[:5]

['.с.,и спросил его:  о Посланник Аллаха!Ты порицаешь что-то из слушания?  Он ответил: я не порицаю ничего из него,но передай им,чтобы они открывали свои собрания Кораном и закрывали их Кораном ...........Это дошедшие до нас мнения и тот кто находится в поисках истины,по мере изучения этого вопроса будет сталкиваться с',
 'Роднее всех родных Попала я в ГКБ №8 еще в декабре 2002 г. Ехать в больницу очень боялась, но делать нечего, рожать надо. О ГКБ № 8 слышала много хорошего. Когда приехала в приемное отделение, мои колени тряслись от страха, но меня там встретили очень вежливо. Осмотр врача был аккуратным',
 'Непорядочное отношение к своим работникам Работаю в сети Доктор Столетов Центр. В моем трудовом договоре прописано, что работад. оплачивает обучение по сертифик. А реально меня принуждают оплачивать половину. Жаль, но наверное придется искать новую работу!  купить китайские сотовые телефоны ',
 '). Отсутствуют нормативы, Госты и прочее, что позволило бы отличить  хорошие правильн

In [6]:
counts = {0 : 0, 1 : 0, 2 : 0,}
for label in list(zip(*dataset_train))[1]:
    counts[label] += 1

print(counts)

{0: 12974, 1: 23928, 2: 13098}


In [7]:
tf_texts = tf.data.Dataset.from_tensor_slices(texts)

In [8]:
from keras.layers import TextVectorization

text_vectorization = TextVectorization(max_tokens=10000,
                                      output_mode="int",
                                      output_sequence_length=50)

text_vectorization.adapt(tf_texts, batch_size=100)

In [9]:
encode = text_vectorization(["Привет, мой дорогой друг! Как ты, блаблабла, yolo?"])
print(encode)

tf.Tensor(
[[2889  190 1521 1222  180  305    1    1    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0]], shape=(1, 50), dtype=int64)


In [10]:
del texts
del tf_texts

In [11]:
inputs = keras.layers.Input(shape=(), dtype="string", name="input")
x = text_vectorization(inputs)
x = keras.layers.Embedding(text_vectorization.vocabulary_size(), 500, name="embedding")(x)
x = keras.layers.LSTM(500, recurrent_dropout=0.1, return_sequences=True, name="lstm_1")(x)
x = keras.layers.LayerNormalization(name="norm_1")(x)
x = keras.layers.LSTM(500, recurrent_dropout=0.1, name="lstm_2")(x)
x = keras.layers.LayerNormalization(name="norm_2")(x)
x = keras.layers.Dense(1024, name="dense_1")(x)
x = keras.layers.LayerNormalization(name="norm_dense_1")(x)
x = keras.layers.ReLU(name="relu_1")(x)
x = keras.layers.Dropout(0.25, name="drop_1")(x)
x = keras.layers.Dense(1024, name="dense_2")(x)
x = keras.layers.LayerNormalization(name="norm_dense_2")(x)
x = keras.layers.ReLU(name="relu_2")(x)
x = keras.layers.Dropout(0.25, name="drop_2")(x)
outputs = keras.layers.Dense(3, activation="softmax", name="output")(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="lstm_classification_model")
model.summary()

Model: "lstm_classification_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None,)]                 0         
                                                                 
 text_vectorization (TextVec  (None, 50)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 50, 500)           5000000   
                                                                 
 lstm_1 (LSTM)               (None, 50, 500)           2002000   
                                                                 
 norm_1 (LayerNormalization)  (None, 50, 500)          1000      
                                                                 
 lstm_2 (LSTM)               (None, 500)               2002000   
                                         

In [12]:
#переведем датасеты в формат tf.data.Dataset
tf_dataset_train = tf.data.Dataset.from_tensor_slices((list(list(zip(*dataset_train))[0]),list(list(zip(*dataset_train))[1]))).batch(50)
tf_dataset_val = tf.data.Dataset.from_tensor_slices((list(list(zip(*dataset_val))[0]),list(list(zip(*dataset_val))[1]))).batch(50)

In [13]:
#обучение будет довольно долгим на средних компьютерах, так что можно просто перейти на следующий блок,
#скачав модель с гитхаба и переместив ее в папку с этим notebook'ом
lr_sched= tf.keras.optimizers.schedules.ExponentialDecay(0.001, 1000, 0.96)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_sched),
             loss=keras.losses.SparseCategoricalCrossentropy(),
             metrics=keras.metrics.SparseCategoricalAccuracy())

history = model.fit(tf_dataset_train, validation_data=tf_dataset_val, epochs=100, shuffle=True,
                    callbacks=[keras.callbacks.EarlyStopping(patience=5, verbose=1), 
                               keras.callbacks.ModelCheckpoint("fitted_text_model.tf", save_best_only=True, verbose=1)], 
                    verbose=1)

Epoch 1/100
1000/1000 [==============================] - ETA: 0s - loss: 1.0871 - sparse_categorical_accuracy: 0.4599
Epoch 1: val_loss improved from inf to 1.05705, saving model to fitted_text_model.tf
INFO:tensorflow:Assets written to: fitted_text_model.tf\assets
1000/1000 [==============================] - 303s 299ms/step - loss: 1.0871 - sparse_categorical_accuracy: 0.4599 - val_loss: 1.0571 - val_sparse_categorical_accuracy: 0.4750
Epoch 2/100
1000/1000 [==============================] - ETA: 0s - loss: 1.0149 - sparse_categorical_accuracy: 0.4676
Epoch 2: val_loss improved from 1.05705 to 0.99970, saving model to fitted_text_model.tf
INFO:tensorflow:Assets written to: fitted_text_model.tf\assets
1000/1000 [==============================] - 294s 294ms/step - loss: 1.0149 - sparse_categorical_accuracy: 0.4676 - val_loss: 0.9997 - val_sparse_categorical_accuracy: 0.4645
Epoch 3/100
1000/1000 [==============================] - ETA: 0s - loss: 0.9667 - sparse_categorical_accuracy: 0.4

In [14]:
model = keras.models.load_model("fitted_text_model.tf")

In [36]:
from IPython.display import Image, display

promt = """На рассвете первые лучи солнца осторожно касались земли, пробуждая всё вокруг. 
           Лёгкий ветерок переносил ароматы цветов, шёпот листвы и обещания нового дня. 
           Мир замирал в тишине, где каждая деталь обретала смысл. 
           В эти мгновения казалось, что сама природа напоминала нам ценить её простоту и красоту.
        """
promt = np.array([promt], dtype=np.str_).reshape(1,-1)

prediction = model.predict(promt)
print(prediction)
prediction = prediction.argmax().item()

if prediction == 0:
    print("Ну, ок! Ноль эмоций -_-")
elif prediction == 1:
    print("Оу, ты явно написал что-то милое. Надеюсь, что это не ошибка системы...")
else:
    display(Image(url="https://avatars.mds.yandex.net/i?id=ed7efab13ea582cfb7ea67af71a64662b54b70f9-11005657-images-thumbs&n=13"))

1/1 [==============================] - 0s 123ms/step
[[0.4103907  0.5462118  0.04339749]]
Оу, ты явно написал что-то милое. Надеюсь, что это не ошибка системы...


In [16]:
#примеры положительного, нейтрального и негативного текстов
promt = """На рассвете первые лучи солнца осторожно касались земли, пробуждая всё вокруг. 
           Лёгкий ветерок переносил ароматы цветов, шёпот листвы и обещания нового дня. 
           Мир замирал в тишине, где каждая деталь обретала смысл. 
           В эти мгновения казалось, что сама природа напоминала нам ценить её простоту и красоту.
        """
promt = """Современные технологии оказывают значительное влияние на повседневную жизнь человека. 
           Сфера применения инноваций охватывает множество областей: от медицины и образования до промышленности и развлечений. 
           Развитие искусственного интеллекта и автоматизации позволяет решать сложные задачи быстрее и эффективнее, 
           однако требует продуманного подхода для минимизации возможных рисков и этических вопросов.
        """
promt = """Слушай, ты ведь можешь лучше. 
           Ты берёшься за интересные вещи, но почему не доводишь их до конца? 
           Перестань откладывать на потом. Это только твоя ответственность. 
           Если не начнёшь действовать сейчас, кто будет за это отвечать? 
           Ты знаешь, что у тебя есть потенциал — так используй его!
        """